In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn import linear_model, metrics, preprocessing
from sklearn.model_selection import KFold, train_test_split
import xgboostdata['response_std'] = data[['response_sms', 'response_viber']].std(axis=1)

import azureml.core
from azureml.core import Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

import sys
sys.path.append('../')
sys.path.append('../scripts/')

In [16]:
# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

# Initialise workspace
ws = Workspace.from_config(auth=interactive_auth)

# Data import 
aml_dataset = Dataset.get_by_name(ws, 'train_ds', version='latest')
data = aml_dataset.to_pandas_dataframe()
data.set_index('CardHolder', inplace=True)

# Save it to the further opeartions
original_columns = data.columns
data_origin = data.copy()

/anaconda/envs/py37_default/lib/python3.7/site-packages/azureml/dataprep/api/dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.12.0 --upgrade
  warnings.warn('Your pandas and pyarrow versions are incompatible. '


In [54]:
data = data_origin.copy()
data = data[[*cool_features, *other_features]]

In [55]:
from importlib import reload
import features.functions
reload(features.functions)

#data = features.functions.preprocessing(data)
data_w_features = data.copy()

In [64]:
index_columns = ['gender', 'main_format', 'children']
data = data_w_features.copy()

# Let's fill NaN in "index_columns"
# data['group'] = data.loc[:, 'group'].replace({'test' : 1, 'control' : 0})
data['gender'] = data.loc[:,'gender'].replace({'М' : 2, 'Ж' : 1, 'Не определен' : 0, None : 0})
data['children'] = data.loc[:,'children'].replace({None : -1})

In [65]:
from scripts.metrics import custom_metric
import features.functions
reload(features.functions)

data = data.reset_index().drop('CardHolder', axis=1)
# data.loc[:, 'group'].replace({1 : 'test', 0 : 'control'}, inplace=True)

data['class'] = 0
data.loc[(data['group'] == 'control') & (data['response_att'] == 1), 'class'] = 1
data.loc[(data['group'] == 'test') & (data['response_att'] == 0), 'class'] = 2
data.loc[(data['group'] == 'test') & (data['response_att'] == 1), 'class'] = 3

train, test = train_test_split(data)
train, test = features.functions.feature_generation(train, test, 
                                mean_columns=['group', 'response_att', 'stdev_days_between_visits_15d'],
                                mean_index_columns=['gender', 'age', 'children'])

x_train = train.drop(['class', 'group', 'response_att'], axis=1)
y_train = train['class']

x_test = test.drop(['class', 'group', 'response_att'], axis=1)
y_test = test['class']

params = {'n_estimators': 20, 'eval_metric': ['merror', 'mlogloss'], 'verbose': True, 'n_jobs': -1}
model = xgboost.XGBClassifier(**params)

model.fit(x_train, y_train, verbose=True, eval_set=[(x_test, y_test)])

pred = model.predict_proba(x_test)
final = pred[:,3] + pred[:,0] - pred[:,1] - pred[:,2]
test['uplift'] = final

custom_metric(test)

ValueError: feature_names must be unique

In [27]:
cool_features = ['response_sms', 'k_var_days_between_visits_15d', 'response_viber',
                'k_var_days_between_visits_1m', 'k_var_days_between_visits_3m',
                'perdelta_days_between_visits_15_30d', 'stdev_discount_depth_1m',
                'stdev_days_between_visits_15d', 'cheque_count_6m_g48', 'cheque_count_12m_g48',
                'k_var_disc_share_3m_g34', 'cheque_count_6m_g40', 'k_var_discount_depth_15d',
                'crazy_purchases_goods_count_12m', 'sale_count_12m_g49', 'cheque_count_12m_g46',
                'k_var_count_per_cheque_3m_g34', 'cheque_count_6m_g32', 'cheque_count_6m_g41',
                'k_var_cheque_15d']
other_features = ['group', 'response_att', 'stdev_days_between_visits_15d', 'gender', 'age', 'children']